# Rust 101

## Ziel
Das Ziel des Tutorials ist es, eine Einführung in Rust zu bekommen, und am Ende einen Compiler und Interpreter von UB++ zu haben. Das ganze Projekt wird am Ende als Github-Repository zur Verfügung gestellt. Hier werden wir Schritt für Schritt versuchen, uns an das Endprodukt anzutasten, und jeweils einzelne wichtige Aspekte von Rust kennen lernen.

## UB++

Wie wir alle wissen, machen wir Geilen Scheiss (TM). Es ist aber viel geilerer Scheiss, wenn der Scheiss auch so geschrieben wird, wie der Schnabel wächst. Entsprechend kann jeder in diesem Tutorial, die Keywords seinem Schnabel nach anpassen.

UB++ ist eine einfache (interpretierte) Sprache, angelehnt an Java, die simple Logik und Berechnungen erlaubt.

```
definier e variable wo priimZahl heisst mit em wert e frog "Was füre Zahl wotsch teste?";
definier e variable wo teiler heisst mit em wert 2;
definier e variable wo primzahlWurzel heisst mit em wert priimZahl hoch 0.5;
definier e variable wo rest heisst mit em wert -1;
definier e variable wo priimZahlGfunde heisst mit em wert wohr;

solang de teiler kliiner oder gliich isch wie primzahlWurzel mach {
    falls de (priimZahl rest teiler) isch gliich wie 0 mach {
        priimZahlGfunde isch falsch;
        stop;
    };
    teiler isch teiler plus 1;
}

falls d priimZahlGfunde mach {
    gib us "S isch e priim zahl";
} suscht {
    gib us "S isch kei priim zahl";
};
```

## Rust installieren

Am einfachsten wird Rust direkt mit dem Bash-Skript von der [Rust-Seite](https://rustup.rs) installiert:

```
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
```

Der Installer sollte durch die Installation führen, wobei die defaults alle in Ordnung sein sollten. 

Sollte Rust schon installiert sein, bietet es sich an, Rust mit folgendem Befehl upzudaten:

```bash
 rustup update
```

## Jupyter installieren

Um dieses Notebook zu benutzen, braucht es [Jupyter](https://jupyter.org/install).

## Rust Jupyter Kernel installieren

Abschliessend wird der Rust-Kernel für die Jupyter-Notebooks benötigt. Dieser kann vie cargo wie folgt installiert werden:

```bash
cargo install evcxr_jupyter 
```
## Cargo

Cargo ist das `all-in-one` Tool der Rust-Entwicklung. 

- `cargo new [--lib] <name>`: Erstellt ein neues Rust-Projekt mit Name "name"
- `cargo build [--release]`: Buildet das Rust-Projekt im aktuellen Ordner (sucht nach einem Cargo.toml)
- `cargo run -- [programm arguments]`: Lässt das Rust-Projekt kompillieren und startet es mit den entsprechenden Argumenten
- `cargo add <dependency>`: Fügt eine Dependency zum Cargo.toml hinzu

### Cargo.toml
Das Cargo.toml beinhaltet wichtige Informationen zum Projekt, wie z.B. die Build-Products, Meta-Daten wie Author, Name usw. aber auch alle Abhängigkeiten. Es gibt viele verschiedene Mölgichkeiten Dependencies anzugeben, z.B. direkt vom lokalen File-System, von einem Git-Repository oder aber von [crates.io](https://crates.io).


<!--
 Copyright (c) 2022 Ubique Innovation AG <https://www.ubique.ch>
 
 This Source Code Form is subject to the terms of the Mozilla Public
 License, v. 2.0. If a copy of the MPL was not distributed with this
 file, You can obtain one at http://mozilla.org/MPL/2.0/.
-->

## Abhängigkeiten

Wir werden in diesem Tutorial [`pest`](https://pest.rs/) benutzen. `pest` bietet uns eine Möglichkeit, die Grammatik für unsere Sprache in einer externen Datei auszulagern, und dann bereits die einzelnen Tokens als Objekte zu erstellen.

Um in Rust Abhängikeiten hinzuzufügen, müssen wir (normalerweise) mit `cargo add <name_der_dependency>` im Rust-Projektordner die Abhängigkeit hinzufügen. Natürlich kann dies auch manuell im `Cargo.toml` gemacht werden. 

Da wir hier aber Jupyter-Notebooks verwenden, müssen wir uns nicht direkt mit dem auseinandersetzen. Folgende zwei Abhängigkeiten werden wir in unserem Projekt benutzen entsprechend müssen wir die unserem Jupyter-Kernel hinzufügen.



In [24]:
:dep pest = "*"
:dep pest_derive = "*"

## Code zum Notebook

Der vollständige Code befindet sich unter `ubpp/src` und ist in drei Steps eingeteilt, die ungefähr der Einführungsreihenfolge im Notebook entsprechen. Falls ein Schritt nicht funktioniert, kann der Code kopiert werden. Er sollte allerdings nur als Hilfe dienen ;).

## Boilerplate

Um nun einen Einstieg für unsere kleine Script-Sprache zu bieten, lassen wir `pest` alle Implementierungen anhand unseres Grammar-Files ableiten (`#[derive]`).

Mit dem `#[grammar]`-Attribut können wir den Pfad zu unserem Grammar-File angeben. In unserem Fall hier, geben wir den absoluten Pfad an, damit das Grammar-File gefunden wird, unabhängig von wo aus unser Rust-Programm kompiliert wird.

Für das Notebook wird, anders als im Code-Beispiel, alles in einer Datei gemacht. Aufspalten zu verschiedenen Modulen etc. ist nicht relevant für dieses Projekt.

In [25]:
use pest::Parser;
use pest::iterators::Pairs;

// Wird für den Jupyter-Kernel gebraucht, mit den neueren Rust-Versionen ist dies allerdings hinfällig.
extern crate pest;
#[macro_use]
extern crate pest_derive;


#[derive(Parser)]
#[grammar = "/Users/patrickamrein/Documents/Ubique/git/introduction-to-rust/ubpp.pest"]
pub struct UBPP;

## Erste Schritte

Da wir nun erstmals die Grammatik unserer Sprache geladen haben, können wir nun schauen, wie unser Code-Beispiel von Oben aussieht. Wir bekommen von  `pest` ein geparsedes File, das schon "Tokenized" ist, und nur  drauf wartet von uns umgewandelt zu werden.

Wir sehen unten ein Beispiel von einem `Multiline-Raw-String-Literal`, das es uns erlaubt, einen String über mehrere Zeilen zu schreiben, aber auch Anführungszeichen ohne escaping zu benutzen. Strings in Rust kann man grundsätzlich immer über mehrere Zeilen schreiben.

Danke dem "derive" Attribut auf unserer Klasse (struct UBPP), wurde vom Rust-Compiler anhand des `Pest-Files` bereits eine Menge von Implementierungen erstellt. Wird können nun diese "statische" Funktion auf der Struct mit folgendem Syntax aufrufen:
```rust
UBPP::parse(Rule::file, primzahl_tester).unwrap();
```

Beachte, dass solche assoziierte Funktionen auch auf Instanzen aufgerufen werden können. Wir werden später sehen, wie wir Funktionen einer Klasse hinzufügen können. Als erstes soll hier angemerkt werden, dass folgender Syntax equivalent ist:

In [ ]:
struct Test {}
impl Test {
    fn test_function(&self) {
        println!("test");
    }
}

let test_instance = Test{};
Test::test_function(&test_instance);
test_instance.test_function();

In [29]:
let primzahl_tester = r#"
definier e variable wo priimZahl heisst mit em wert e frog "Was füre Zahl wotsch teste?";
"#;
let parse_result = UBPP::parse(Rule::file, primzahl_tester).unwrap();

Wir sehen hier, das "unwrap", das benutzt wird. Die `parse` Funktion liefert uns nämlich ein `Result`-Objekt zurück, auf welchem wir keine weiteren Opperation des eigentlichen Typen ausführen können. Um an das innere Objekt zu kommen, müssten wir ein Error-Handling einbauen (sehen wir später), oder wir wählen wie hier den einfachen (und während der Entwicklung durchaus legtimen) Weg fes force-unwrap.

Doch wie können wir jetzt sehen, was geparsed wurde? Rust kennt zwei verschiedene "Anzeige" Arten. Beide der Anzeige-Arten werden als "Traits" implementiert (ähnlich wie Interfaces resp. Protocols oder wie sie sonst heissen), und können für jeden Typen implementiert werden.

Zum einen wäre das Trait `Display`, welches am Ende eine `to_string` Funktion zur Verfügung stellt. Dies kann benutzt werden um eine Standard-Darstellung eines Typens zu definieren. 

Die zweite Möglichkeit ist  `Debug` und ermöglicht eine erweiterte Darstellung eines Typen für Debugging-Zwecke. Das `Debug`-Trait kann ebenfalls für jeden Typen "derived" werden, was dazuführt, dass jedes Feld mit Name und Wert angezeigt wird.

Wir vergleichen nun mal die zwei Arten mit dem Parse-Result.

Für die Ausgabe verwenden wir das `println`-Macro, welches uns "Compile-Time-Format-Strings" gibt. zwei geschweifte Klammern werden benutzt um die `to_string` Funktion eines Typs zu benutzen. Mit ":?" innerhalb der Klammern können wir Rust auffordern, die Debug-Implementierung aufzurufen. Wenn man zusätzlich noch einen "#" vor das Fragezeichen setzt, vordert man einen `pretty-debug-print`.

Es gibt noch weitere Format-String modifiers, auf welche wir hier jetzt nicht weiter eingehen.

In [ ]:
println!("{:?}", parse_result);
// println!("{:#?}", parse_result);
// println!("{}", parse_result);

## Das AST-Model (Step1)

Jetzt wo wir eine von `pest` geparsed Struktur haben, wollen wir unsere eigene Struktur aufbauen, die anschliessend relativ einfach ausgeführt werden kann (Step2). Wir nennen diese Struktur AST (Abstract Syntax Tree) und sie representiert unser Program im Speicher.

Wir werden nun versuchen, nach und nach die verschiedenen Klassen (Structs) und Enums aufzubauen. Wir fangen mit dem tiefsten Element an (Atomics) und bauen immer komplexere Strukturen, die aus diesen Bauteilen zusammengesetzt sind.

> Wir werden jetzt verschiedene Enums/Structs und so weiter einfügen. Da wir mit selbstreferenzierten Typen arbeiten musst du mit Code ausführen warten, bis wir alle Teile beisamen haben.


## Expressions
### Atomics

In unserer Sprache wird es fünf verschiedene "Atomic"-Datatypes geben. Wir werden mit alle Zahlen-Typen als "double" (64bit Floating-Point) darstellen, was das ganze Handling vereinfacht, und für unseren Use-Case ausreichend ist. Wir nennen diesen Datentypen `Number`. Weiter wird es einen `String`, einen `Bool`, einen `Null` und einen `Interrupt` Atomic geben.

Der `Interrupt` wird ein Atomic sein, um ein frühzeiteges Ende einer While-Schlaufe erkennen zu können (und ist etwas ein Hack, aber who cares, wir sind ja die Language-Architects).

Wir werden viele der Typen als Rust-Enums handeln. Der Rust-Enum Typ ist mehr als eine blosse Enumeration, sondern kann auch komplexe Objekte halten.

Doch vorerst wollen wir kurz auf die verschiedenen Grund-Datentypen von Rust eingehen:

- `u8/u16/u32/u64 | i8/i16/i32/i64`: Dies sind die Integer Datentypen in Rust. Der Buchstabe gibt jeweils an, ob der Typ "*u*nsigned" ist oder signed. Die Zahl gibt die jeweilige Bit-Breite an, also für ein Byte zum Beispiel "u/i8". Je nach Platform wird bis `i/u128` unterstützt.

- `f32/f64`: Sind die Fliesskommadatentypen. Die Zahl stellt wiederum die Bitbreite an, also f32 für single Precision und f64 für double precision.

- `bool`: Der übliche Boolean-Datentyp.

- `String`: Ist ein Heap allozierter veränderbarer String-Typ. Strings in Rust _müssen_ immer gültiges UTF-8 sein.

- `&str`: Ist ein "geliehener" String, respektive "String-Slice". Wir wollen hier vorerst nicht genauer auf Borrows resp. den Unterschied von `String` und `&str` eingehen.

Wir haben hier bewusst auf komplexere Typen wie Arrays, Maps, Synchronisations-Typen und ähnliches verzichtet. Im verlaufe des Tutorials werden wir uns noch mit Arrays und Maps auseinandersetzen.

Es gibt drei verschiedene Typen von Enum-Variants:

- Unit-Type: Der Unit-Type ist sehr ähnlich zum klassischen C enum Typ. Er stellt einfach eine "Zahl" oder Enumeration dar

- Tuple-Type: Der Tuple-Type ist, anders als der Unit-Typ, fähig ein oder mehrere Objekte zu halten. Diese werden einfach als Tuple übergeben, und haben an sich keinen Namen, sondern bloss eine festgelegte Reihenfolge

- Struct-Type: Struct-Type hält eine Struct (also eine Rust-Klasse) und hat benannte Felder.


Folgend sehen wir das Gerüst für unseren Atomic-Datatype. Es fehlt allerdings noch der Boolean Typ. Füge ihn hinzu.

In [ ]:
#[derive(Debug, Clone)]
pub enum Atomic {
    String(String),
    Number(f64),
    /* Füge hier noch einen weiteren Typen, nämlich den Boolean Typ ein */
    Null,
    Interrupt
}


> Wir sehen das Attribut `#[derive(Debug, Clone)]` über dem Enum. Was bedeutet das? Wir sehen hier das "derive" Attribut. Dieses Attribut dient dazu, default Implementierungen für bestimmte Typen zur Verfügung zu stellen. Dieses Derive Macro ist ein sogenanntes "Procedural Macro" und erlaubt es Rust-Code während dem resp. vor dem Kompilieren auszuführen. In unserem Falle werden Implementierungen für den Debug-Output und für die "Deep-Copy" erstellt.


In [ ]:
/// Unser DatenType für die beiden logischen Operatoren && und ||
#[derive(Debug, Clone)]
pub enum LogicOp {
    And(Expression, Expression),
    Or(Expression, Expression),
}

/// Unser DatenTyp für Vergleiche < > <= >= == und !=
#[derive(Debug, Clone)]
pub enum Comparison {
    Smaller(Expression, Expression),
    SmallerEquals(Expression, Expression),
    Equals(Expression, Expression),
    Greater(Expression, Expression),
    GreaterEquals(Expression, Expression),
}

### Binäre (numerische) Operationen

Um die Grundoperationen darzustellen, verwenden wir einen Struct-Type-Enum. Wir werden dann den Unterschied im Pattern-Matching erkennen können, und was für Vor-/Nachteile es mit sich bringt.

Auch hier fehlt unsere letzte Variant, die Pow-Variant. Füge sie hinzu.

In [ ]:
#[derive(Debug, Clone)]
pub enum BinaryOp {
    Plus { left: Expression, right: Expression },
    Minus { left: Expression, right: Expression },
    Mul { left: Expression, right: Expression },
    Div { left: Expression, right: Expression },
    Mod { left: Expression, right: Expression },
    /* Füge hier den Pow-Variant ein */
    None,
}

### Conditional Expression

In unserer Sprache wird es eine If-Expression haben. Die If-Expression returned jeweils die letzte Expression als Wert, und braucht entsprechend zwingend einen Else-Branch.

In unserem AST stellen wir diese If-Expression als Struct dar. Eine Struct in Rust ist ähnlich einer Klasse in anderen Sprachen. Sie wird mit dem Keyword `struct` definiert. Ähnlich wie in Kotlin werden anschliessend die Feldernamen und der entsprechende Typ aufgezählt. Eine Struct in Rust besitzt allerdings an sich keine Funktionen und entsprechend gibt es keine Möglichkeit Default-Values anzugeben (es gibt allerdings den "Default"-Trait, auf den wir hier aber nicht weiter eingehen).

Eine Struct kann anschliessend mit dem Struct-Initializer-Syntax erstellt werden. Das kleine Beispiel-Listing kannst du ausführen und damit rumspielen.

In [12]:
#[derive(Debug)]
struct DiesIstEineStruct {
    sie_hält_einen_string: String,
    und_einen_integer: u32
}

// erstellen kann man sie nun wie folgt:

println!("{:?}", 
    DiesIstEineStruct {
        sie_hält_einen_string: "Dies ist ein string".into(),
        und_einen_integer: 3
    });


DiesIstEineStruct { sie_hält_einen_string: "Dies ist ein string", und_einen_integer: 3 }


In [ ]:
#[derive(Debug, Clone)]
pub struct ConditionalExpression {
    pub condition: Box<Expression>,
    pub body: Vec<Token>,
    pub body_expression: Box<Expression>,
    /* Fügen jeweils else_body und else_body_expression wie für den true branch ein */
}

### Heap und Stack

In Rust wird versucht möglichst viel auf dem Stack zu stellen. Wer sich mit Memory-Layout etwas auskennt, wird wissen, dass, um ein Objekt auf dem Stack zu plazieren, die grösse während dem Kompilieren bekannt sein muss. Für die meisten "Grund-Datentypen" ist dies erfüllt. Wir bekommen allerdings ein Problem, sobald wir eine Struktur haben, die sich selbst wieder Referenziert. 

Da wir mit nested Expressions arbeiten werden, z.B. `(3+2+3) * (4+4+6)`, was eine Expression ist die aus zwei (resp. mehr) SubExpressions besteht, führt dies unweigerlich zum Problem, dass die Grösse des Datentyps prinzipiel nicht beschränkt ist. Wir können also im Voraus nicht wissen, wieviele dieser Sub-Expressions auf den Stack müssen, und darum können wir den benötigten Speicher nicht bereitstellen.

Einen Ausweg gibt es, sobald wir Heap-Allozierte Daten-Typen haben. Dies sind meist dynamisch wachsende Typen, wie z.B. Arrays, Maps, Strings (oder eben selbstreferenzierende Strukturen). Bei einem Heap-Alloziertem Typen müssen wir nicht mehr die Struktur auf dem Stack speichern, sondern nur noch eine Referenz auf den entsprechenden Bereich im Heap. Dies löst unser Probelm, mit dem Nachteil, dass wir an "Geschwindigkeit" verlieren, da Zugriffe auf den Heap teurer sind als solche, die nur auf den Stack verweisen.
### Box
Im Beispiel oben verwenden wir den Datentyp `Box<T>`, der einen generischen Typen darstellt, welcher den Datentypen `T` auf dem Heap alloziert, und als Pointer interpretiert wird. Allozierung und Deallozierung werden automatisch von Rust gehandelt, so dass der Speicher freigegeben wird, sobald die `Box<T>` aus dem Scope verschwindet.

Vergleiche nachfolgend die Speichergrösse für einen Boxed Typ und den Typen auf dem Stack. Wir sehen dass der Typ `SizeOfStruct` nur einen Integer hält (4 Bytes) und entsprechend auch nur 4 Bytes gross ist. Der Boxed Type ist abhängig von der zugrundeliegenden Plattform, heutzutage in der Regel 64bits sprich 8 Bytes.

In [16]:
struct SizeOfStruct {
    test: u8
}
println!("{}",std::mem::size_of::<SizeOfStruct>());
println!("{}", std::mem::size_of::<Box<SizeOfStruct>>())

1
8


()

### Arrays (Vec)

Ein weiterer wichtiger Datentyp stellen Arrays dar. Die einfachste Art einen Array in Rust abzubilden ist mit dem `Vec<T>` Datentyp. Dieser ist wiederum generisch zum Typen T. Wie bereits erwähnt wird ein Vec stets auf dem Heap alloziert. Da der Vector nur einem Typen `T` zu geordnet werden kann, muss in bestimmten Fällen der Typ explizit mit angegeben werden. Dies ist i.d.R. der Fall, wenn der Type-Inferer es nicht schafft, einen eindeutigen Typen zu finden. Meist passiert dies mit den `collect` Funktionen, welche auf Iterators definiert sind.

In solchen Fällen kann entweder der Typ direkt bei der Variablen-Deklaration angegeben werden, oder aber mit dem sogenannten Turbo-Fish.

Beachte nun zum Beispiel die zwei Vektoren, und anschliessend, der Gebrauch des Turbo-Fischs bei der `parse`-Funktion.

In [8]:
let array : Vec<u32> = vec![];
let some_elements = vec![0;3];

println!("{:?}", array);
println!("{:?}", some_elements);
println!("{}", "3.2".parse::<f32>().unwrap());

[]


[0, 0, 0]
3.2


### Options

In Rust sind alle Typen immer initialisiert (sprich niemals Null). Es gibt aber Situationen, in denen ein Wert vorhanden, oder eben nicht vorhanden sein kann oder muss.

Falls das ein Resultat einer Operation ist, kann man z.B. den `Result<T>` Datentypen verwenden, den wir später noch kennenlernen werden. Für Structs oder Enums empfiehlt sich der `Option<T>` Datentyp, der im Wesentlichen folgendes Enum darstellt (natürlich noch erweitert mit vielen hilfreichen Funktionen aus der Standardbibliothek).

In [9]:
#[derive(Debug)]
enum CustomOption<T> {
    Some(T),
    None
}

println!("{:?}", CustomOption::Some(0u32));
println!("{:?}", CustomOption::<u32>::None);


Some(0)
None


Unser erster Schritt, das Definieren des ASTs, ist nun abgeschlossen und im folgenden sind die oben erwähnten Strukturen ergänzt mit den Statement Strukturen.

Gehe nochmals durch den Code, und führe ihn anschliessend aus.

In [26]:
#[derive(Debug, Clone)]
pub enum Atomic {
    String(String),
    Number(f64),
    Bool(bool),
    Null,
    Interrupt
}

#[derive(Debug, Clone)]
pub enum LogicOp {
    And(Expression, Expression),
    Or(Expression, Expression),
}


#[derive(Debug, Clone)]
pub enum Comparison {
    Smaller(Expression, Expression),
    SmallerEquals(Expression, Expression),
    Equals(Expression, Expression),
    Greater(Expression, Expression),
    GreaterEquals(Expression, Expression),
}

#[derive(Debug, Clone)]
pub enum BinaryOp {
    Plus { left: Expression, right: Expression },
    Minus { left: Expression, right: Expression },
    Mul { left: Expression, right: Expression },
    Div { left: Expression, right: Expression },
    Mod { left: Expression, right: Expression },
    Pow { left: Expression, right: Expression },
    None,
}

/// Expressions
#[derive(Debug, Clone)]
pub struct ConditionalExpression {
    pub condition: Box<Expression>,
    pub body: Vec<Token>,
    pub body_expression: Box<Expression>,
    pub else_body: Vec<Token>,
    pub else_body_expression: Box<Expression>,
}


#[derive(Debug, Clone)]
pub enum Cast {
    String(Expression),
    Int(Expression),
    Bool(Expression),
}

#[derive(Debug, Clone)]
pub enum Expression {
    Atomic(Atomic),
    Ident(String),
    LogicOp(Box<LogicOp>),
    Comparison(Box<Comparison>),
    BinaryOp(Box<BinaryOp>),
    Conditional(ConditionalExpression),
    Input(Box<Expression>),
    Cast(Box<Cast>),
}

#[derive(Debug, Clone)]
pub enum Statement {
    VariableAssignment(VariableAssignment),
    Conditional(Conditional),
    Expression(Expression),
    Print(Expression),
    Loop(Loop)
}

#[derive(Debug, Clone)]
pub struct Loop {
    pub condition: Box<Expression>,
    pub body: Vec<Token>,
}

#[derive(Debug, Clone)]
pub struct Conditional {
    pub condition: Box<Expression>,
    pub body: Vec<Token>,
    pub else_body: Option<Vec<Token>>,
}

#[derive(Debug, Clone)]
pub struct VariableAssignment {
    pub new_definition: bool,
    pub ident: String,
    pub value: Expression,
}

#[derive(Debug, Clone)]
/// Unsere Sprache besteht aus verschiedenen Tokens
pub enum Token {
    /// Eine Expression ist ein Token, das einen Wert darstellt. Enstprechend kann eine Expression z.B. einer Variable 
    /// hinzugefügt werden.
    Expression(Expression),
    /// Ein Statement führt Code aus, stellt aber keinen Wert dar und kann somit nur alleine stehen
    Statement(Statement),
    /// Vorzeitiges Ende aus einer While-Loop
    Break,
    /// Placeholder für potenzielles early return
    Return
}

## AST Erstellen (Step 2)

Wir besitzen nun alle Strukturen, die wir benötigen um unseren AST darzustellen. Im nächsten Schritt wollen wir das geparsede Resultat von `pest` in eine etwas einfacher zu brauchenden, typisierte Struktur umwandeln.

Wir werden nun im `step2.rs` versuchen, rekursive Funktionen aufzubauen, die wir verwenden können um unsere Strukturen zu befüllen.

Pest liefert uns jeweils `Pair<Rule>`, wobei `Rule` einem Typen unserer Grammatik entspricht.

### Kontrollstrukturen

In Rust gibt es verschiedene Kontrollstrukturen. Es gibt die klassischen `if` und `while` Statements, die basierend auf einer Condition, einen entsprechenden Branch ausführen.

In [11]:
let mut  i_am_boolean = true;
if i_am_boolean {
    println!("he is boolean");
} else {
    println!("he is not :(");
}

let mut i = 0;
while i < 3 {
    println!("{}", i);
    i+=1;
}

he is boolean
0
1
2


()

Weiter gibt es eine `loop`-Struktur, die einer `while (True) {}` Schlaufe entspricht (sprich nur mit dem Keyword `break` gestoppt werden können).


In [12]:
let mut i = 0;
loop {
    if i > 5 {
        break;
    }
    i += 1;
}
println!("i = {}", i);

i = 6


<!--
 Copyright (c) 2022 Ubique Innovation AG <https://www.ubique.ch>
 
 This Source Code Form is subject to the terms of the Mozilla Public
 License, v. 2.0. If a copy of the MPL was not distributed with this
 file, You can obtain one at http://mozilla.org/MPL/2.0/.
-->

### Pattern Matching

Eine spezielle Struktur stellt das Pattern-Matching dar. Pattern-Matching, wie der Name beretis andeutet, kann benutzt werden, um bestimmte Variabeln z.B. auf bestimmte Enum-Varianten zu matchen. Pattern-Matching mit `match` muss immer 'exhaustive' sein. Für Wildcard patterns kann das Konstrukt `_ => {}` verwendet werden.

Im Pattern-Matching können die Felder von den Enum-Variants direkt an Variabeln gebunden werden. Weiter können Guards oder verschachtelte Patterns benutzt werden um das Pattern einzuschränken.

Einzelne Statements können auch auf einer Linie ohne `{}` geschrieben werden, wobei sie dann aber als 'Expressions' interpretiert werden (und entsprechend muss jeder Branch den gleichen return wert haben).

In [13]:
pub enum Test {
    Int(i32),
    Bool(bool),
    String(String)
}

let an_int = Test::Int(34);
match an_int {
    Test::Int(the_int) if the_int == 234 => {
        println!("234")
    }
    Test::Int(314) => {
        println!("fast pi")
    }
    Test::Int(1..=32) => {
        println!("irgendwo zwischen 1 und 32")
    }
    Test::Int(-32..=-1) => {
        println!("irgendwo zwischen -1 und -32")
    }

    Test::Int(the_int) => {
        println!("It is an int with value: {}", the_int)
    }
    Test::Bool(the_bool) => println!("{}", the_bool), // hier brauchts ein ','
    _ => {
        panic!("this should not happen");
    }
};

It is an int with value: 34


Expressions können ebenfalls direkt mit dem `if-let`-Statement gematched werden, falls nur eine spezielle Variante geprüft werden will.

In [4]:
// Copyright (c) 2022 Ubique Innovation AG <https://www.ubique.ch>
// 
// This Source Code Form is subject to the terms of the Mozilla Public
// License, v. 2.0. If a copy of the MPL was not distributed with this
// file, You can obtain one at http://mozilla.org/MPL/2.0/.

if let Test::Int(i) = an_int {
    println!("Yes an int {}", i);
}

Yes an int 34


()

Da wir uns jetzt etwas mit Schlaufen und Pattern-Matching auskennen, fahren wir folgend fort. Wir wollen mit einem `body` Rule (also unserem Top-Level Match anfangen) und schreiben jeweils Funktionen um die Sub-Patterns in den AST umzuwandeln.

Wir wollen also Top-Level auf drei Patterns matchen:

- Break
- Expressions
- Statements

Da wir nur wenige Statements haben, inlinen wir das Sub-Pattern direkt im Schlaufen-Body. Für die Expressions benutzen wir eine Separate-Funktion.

<!--
 Copyright (c) 2022 Ubique Innovation AG <https://www.ubique.ch>
 
 This Source Code Form is subject to the terms of the Mozilla Public
 License, v. 2.0. If a copy of the MPL was not distributed with this
 file, You can obtain one at http://mozilla.org/MPL/2.0/.
-->

### Funktions-Definitionen



In Rust werden Funktionen mit dem `fn` Keyword definiert. Wie bei den Structs kann der Modifier `pub` verwendet werden, um die Funktion ausserhalb des Moduls zur Verfügung zu stellen. Weiter werden die Argumente gemäss `<name> : <Type>>` in Klammern aufgezählt. 

Es gibt drei verschiedene `modifiers` für die Argumente. Diese können die `Ownership` übernehmen, sprich sind der alleinige Besitzer eines Wertes, können einen Wert ausleihen (mehrere Orte können gleichzeitig von einem Wert leihen, aber es gibt jeweils immer nur einen Owner), oder aber einen Wert modifizierbar ausleihen. Der `Borrow` wird jeweils mit dem Ampersand ('&') dargestellt und mit `mut` modifiziert.

Der Rückgabewert wird nach einem Pfeil nach der Klammer der Argumente mit angegeben.

Beachte folgende Beispiele:


> Der Rust-Compiler ist dein Freund. Hör auf ihn, wenn er dir versucht was zu erzählen, da oft der richtige Lösungsweg vorgeschlagen wird, resp. weitergehende Informationen zum Problem angezeigt werden.
Versuch folgendes Beispile zum Kompillieren zu bringen.


In [11]:
// Das ist Owned aber nicht mutable
fn owned(the_string: String) -> String {
    the_string.push_str("test");
    the_string
}
// Das ist nur geborgt
fn borrowed(the_string: &String) {
    // dies geht nicht, weil wir keinen mutable Access auf the_string haben
    the_string.push_str("test");
    println!("{}", the_string);
}
//Das ist geborgt, aber mutably
fn mut_borrowed(the_string: &mut String) {
    the_string.push_str("test");
}

let my_string = String::new();
let _ = owned(my_string);
borrowed(&my_string);
let mut my_string = my_string;
mut_borrowed(&mut my_string);

println!("{}", my_string);

Error: cannot borrow `the_string` as mutable, as it is not declared as mutable

Error: cannot borrow `*the_string` as mutable, as it is behind a `&` reference

Error: borrow of moved value: `my_string`

In [ ]:
pub fn parse_body(body: Pair<Rule>) -> Vec<Token> {
    // wir definieren einen leeren array mit Tokens.
    // Diese werden anschliessend befüllt und zurückgegeben.
    let mut tokens = vec![];
    // Hier sehen wir eine for loop
    // die `into_inner` Funktoin vom Pair liefert uns die möglichen inneren Regeln.
    // In unserem Fall beinhaltet der body statements/expressions oder Break...
    for pair in body.into_inner() {
        // ... entsprechend machen wir hier ein sogenanntes `Pattern matching`
        match pair.as_rule() {
            Rule::break_keyword => tokens.push(Token::Break),
            // für Statements inlinen wir die Sub-Patterns
            Rule::statement => {
                // Unwrap hier ist Safe, weil wir wissen, dass an dieser Stelle 
                // innere Regeln existieren müssen (da sonst unserer Grammatik widersprochen würde)
                let inner = pair.into_inner().next().unwrap();
                // Match Strukturen sind Expressions und können entsprechend ein Resultat returnen
                // allerdings muss jeder Branch den gleichen Return-Wert haben!
                let stmt = match inner.as_rule() {
                    Rule::variable_statement => as_var_assignment(inner),
                    Rule::if_statement => as_if_statment(inner),
                    Rule::print_statement => as_print_statement(inner),
                    Rule::expression_statement => Token::Statement(Statement::Expression(
                        as_expression(inner.into_inner().next().unwrap()),
                    )),
                    Rule::while_statement => as_while_statement(inner),
                    _ => continue,
                };
                tokens.push(stmt);
            }
            Rule::expression => {
                tokens.push(Token::Expression(as_expression(pair)));
            }
            // wir können mit bestimmten makros unsere intention zeigen.
            // so gibt es z.B. unreachable!() aber auch todo!() oder unimplemented!()
            // Die Makros nehmen jeweils einen String, der als Beschreibung beim Fehler angezeigt wird.

            // In unserem Falle haben wir alle möglichen Optionen gemäss unserer Grammatik implementiert,
            // entsprechend sind alle anderen Branches nicht erreichbar
            _ => unreachable!(),
        }
    }
    tokens
}

### Lambdas

Lambdas sind "1st Class Citizens" in der Rust-Welt. Sprich jedes Lambda entspricht einem bestimmten Typen, und sie können als Funktionsargumente und in variabeln gespeichert werden. Sie können auch Variabeln aus dem Scope "capturen", wobei jenachdem die `move` Semantik benutzt werden muss (wie bereits gelernt, kontrolliert der Compiler den alleinigen mutable Zugriff auf Variabeln.).

#### Lifetimes

Um dies besser zu verstehen, müssen wir auf das Konzept von "Lifetimes" eingehen. Der Rust Compiler versucht Zugriffe auf Speicherbereiche, die z.B. bereits aufgeräumt wird zu vermeiden. Um dies zu ermöglichen, braucht der Compiler einen Kontext resp. Hinweis über wie lange bestimmte Variabeln "leben". Dies geschieht in Rust (meistens) Implizit mit "Typenähnlichen Generic Arguments". Lebenszeiten werden mit vorangestelltem einfachem Hochkommata deklariert. Dies ist allerdings nur in speziellen Fällen nötig, wo der Compiler keine Eindeutige Lebenszeit ableiten kann.

Es gibt die spezielle Lebenszeit `'static`, welche bedeutet, dass die Referenz während dem ganzen Programmablauf verfügbar ist. Lebenszeiten können stets auf kürzere Zeiten gecasted werden, jedoch nie auf längere (da sonst der Compiler nicht garantieren kann, dass die Variable nicht nach dem freigeben von Speicher benutzt wird).

Folgend ein Beispiel mit zwei Lebzeiten. Hör auf den Compiler und fix das Problem:

In [18]:
// Funktion mit zwei Lebzeiten. `second` lebt länger als `first`, was durch den lifetime bound angegeben wird.
fn lifetimes<'first, 'second: 'first>(_lives_longer_than: &'second str, i_do: &'first str) -> &'second str {
    i_do
}

let l = lifetimes("test", "other");
println!("{}", l);

Error: lifetime may not live long enough

Wenn wir nun wieder zu den Lamdas zurück gehen, müssen wir nochmals auf den Scope zu sprechen kommen. Falls eine Funktion z.B. weitergegeben wird, sprich den aktuellen Scope verlässt, müssen wir garantieren, dass die Variabeln die gecaptured wurden, solange leben können. Dies geht i.d.R. nur, wenn die Variable vom aktuellen Scope in den Scope des Lambdas gemoved wird. Dies wird wie bereits erwähnt mit dem `move` Keyword erreicht.

Lambdas resp. Closures sind im Prinzip nur Pointers zu einem Speicherbereich. Um nun sicherzustellen, dass die Funktion lange genug lebt, müssen wir sie auf dem Heap allozieren, und brauchen entsprechend eine Indirektion via einer `Box<T>`.

Im folgenden Beispiel kombinieren wir Lebenszeiten (welche einfach Generic-Bounds darstellen), mit Closures. Versuch den Compiler zu verstehen, und fixe folgende Funktion:

In [22]:
fn some_function<'lebenszeit>(arg: &'lebenszeit str) -> Box<dyn Fn() -> bool> {
    let a_variable = String::from("Ich bin auf dem Heap");
    let unser_lambda_braucht_move = move || a_variable == arg;
    Box::new(unser_lambda_braucht_move)
}
println!("{}", some_function("Ich bin auf dem Heap")());
println!("{}", some_function("Ich nicht")());

Error: lifetime may not live long enough

Im folgenden Beispiel benutzen wir sogenanntes [`Precidence Climbing`](https://en.wikipedia.org/wiki/Operator-precedence_parser) um unsere numerischen Operationen aufzulösen. `Pest` liefert uns den nötigen Algorithmus, wir müssen nur noch das Mapping von `pest`-Rule zu unserem AST durchführen.

Weiter braucht `pest` noch eine Reduktion zu einem `atomic` Typen. Für diese beiden Operationen benutzen wir die Closures `primary` und `infix`.

In [ ]:
pub fn evaluate_num_operations(pair: Pair<Rule>) -> Expression {
    // Unser precidence climbing höhere Branches stellen höheren Rang dar.
    // In unserem Beispiel ist die Exponential-Funktion rechts assoziativ (bindet nach rechts) und hat die höchste Gewichtung
    // gefolgt von * / und % und
    // mit + und - die am schwächsten bindenden Funktionen

    // Idealerweise würden wir hier den PrecClimber nicht jedesmal bei einer evaluierung neu erstellen, sondern global speichern und benutzen. 
    // Aber für unseren Fall ist das nicht nötig, da der Runtime-Impact wohl vernachlässigbar klein ist.
    let climber = PrecClimber::new(vec![
        Operator::new(Rule::plus, Assoc::Left) | Operator::new(Rule::minus, Assoc::Left),
        Operator::new(Rule::mul, Assoc::Left)
            | Operator::new(Rule::div, Assoc::Left)
            | Operator::new(Rule::mod_op, Assoc::Left),
        Operator::new(Rule::pow, Assoc::Right),
    ]);
    consume_bin_num_op(pair, &climber)
}

fn consume_bin_num_op(pair: Pair<Rule>, climber: &PrecClimber<Rule>) -> Expression {
    // Wir benutzen unsere rekursive Funktion um Rules zu Atomics in einem mathematischen Ausdruck zu reduzieren.
    let primary = |pair| consume_bin_num_op(pair, climber);

    // Unsere infix Closure reduziert eine Operator Rule zu einer BinaryOp expression in unserem Ast.
    // Hier initialisieren wir die Struct-Enum-Variabt mit den Feldern `left` und `right.
    let infix = |left: Expression, op: Pair<Rule>, right: Expression| match op.as_rule() {
        Rule::plus => Expression::BinaryOp(Box::new(BinaryOp::Plus { left, right })),
        Rule::minus => Expression::BinaryOp(Box::new(BinaryOp::Minus { left, right })),
        Rule::mod_op => Expression::BinaryOp(Box::new(BinaryOp::Mod { left, right })),
        Rule::mul => Expression::BinaryOp(Box::new(BinaryOp::Mul { left, right })),
        Rule::div => Expression::BinaryOp(Box::new(BinaryOp::Div { left, right })),
        Rule::pow => Expression::BinaryOp(Box::new(BinaryOp::Pow { left, right })),
        p => unreachable!("{:?}", p),
    };

    // es gibt drei Möglichkeiten um an diesem Punkt zu enden
    match pair.as_rule() {
        Rule::binary_num_expression => climber.climb(pair.into_inner(), primary, infix),
        Rule::parent_expression => consume_bin_num_op(pair.into_inner().next().unwrap(), climber),
        Rule::rvalue_maybe_numeric => get_literal(pair.into_inner()),
        p => unreachable!("{:?}", p),
    }
}

### Iterators

Rust besitzt viele Aspekte einer funktionalen Programmiersprache. Eines der interessanten Konzepte sind die Iterators (Streams, Enumerable, oder wie auch immer), die auf Listen-Typen (Maps, Vectors) aber auch auf Result oder Option definiert sind. Ein Iterator ist im Prinzip ein Trait, das auf einem Typen definiert ist und eine Funktion `next` besitzt. Diese Funktion liefert `Some(T)` falls es eine weiteres Element besitzt oder `None` falls nicht zurück.

Der Standardweg in Rust um von einem Collection-Typen zu einem Iterator zu kommen geht über eine der drei verschiedenen Funktionen, wobei der einzige Unterschied die Art und Weise der "Referenz" auf die Typen ist:

- `iter()`: Die Elemente des Collection-Typen werden immutable geliehen. Man kann Elemente in neue Typen mappen, filtern usw. aber die Elemente selbst nicht verändern.

- `iter_mut()`: Die Elemente des Collection-Typen werden *mutable*  geliehen. Man kann das gleiche erreichen wie bei `iter()` plus zusätzlich die Elemente *in place* bearbeiten. Der Collection-Typ muss mutable sein.

- `into_iter()`: Consumed den Collection-Typen und nimmt Ownership. Man kann alles mit dem Typen machen, der Collection-Typ muss aber geowned sein.

Das `for`-Konstrukt in Rust benutzt automatisch, gemäss Kontext eine der drei Funktionen. Standardmässig wird `into_iter()` aufgerufen und enstprechend sind alle Elemente geowned. Mit `&` resp. `&mut ` kann man die entsprechenden `iter()` oder `iter_mut()` enforcen.

In [40]:
let test = vec![String::from("1"), String::from("2"), String::from("3")];

for t in test {
    println!("{}", t);
}
// warum geht das hier nicht?
let i = &test[0];

let mut other_test = vec![String::from("mut 1"), String::from("mut 2"), String::from("mut 3")];

// was fehlt damit das geht?
for t in &mut other_test {
    t.push_str("... wurde geändert");
}
for t in &other_test {
    println!("{}", t);
}



Error: The variable `i` contains a reference with a non-static lifetime so can't be persisted

Error: borrow of moved value: `test`

Auf den Iterators gibt es viele spannede Funktionen wie:

- `filter`: Nimmt eine Closure und yielded nur die Elemente die, die Closure erfüllen

- `map`: Mapped ein Element in ein anderes

- `for_each`: Funktionale Variante der `for` Loop

- `collect<T>`: Sammelt alle Elemente in einen Collection-Typen. Oft als `collect::<Vec<_>>()` oder `collect::<HashMap<_,_>>()` gebraucht (beachte die Underscores sind valider Syntax, da der Compiler i.d.R. aus dem Inhalt des Iterators bestimmen kann was die enstprechenden Generic-Argumente sein müssen). Anstelle des Turbo-Fish kann man auch das Resultat in einer Variable binden und den Typen explizit mit angeben.

- `cloned()`: Nimmt jedes Element im Iterator und führt `clone()` aus, damit der Resultierende Iterator die Elemente `owned`


In [ ]:
let initial = vec![1, 2, 3, 4, 5, 6];
let plus_one = initial
    .iter()
    .map(|element| element + 1)
    .collect::<Vec<_>>();
let only_even = initial
    .iter()
    .filter(|&element| element % 2 == 0)
    .collect::<Vec<_>>();

for (initial, plus_one) in initial.iter().zip(plus_one.iter()){
    println!("{} -> {}", initial, plus_one);
}

for only_even in only_even {
    println!("{} is even", only_even);
}

// warum funktioniert das?
for skipped in initial.iter().skip(1).step_by(2) {
    println!("{} is also even", skipped);
}

Jetzt sind wir ready, um alle Objekte zu parsen und in einen AST zu bauen. Schau dir folgenden Code an und führe ihn anschliessend aus!

In [27]:
use pest::{
    iterators::{Pair, Pairs},
    prec_climber::{Assoc, Operator, PrecClimber},
};

pub fn parse_body(body: Pair<Rule>) -> Vec<Token> {
    let mut tokens = vec![];
    for pair in body.into_inner() {
        match pair.as_rule() {
            Rule::break_keyword => tokens.push(Token::Break),
            Rule::statement => {
                let inner = pair.into_inner().next().unwrap();
                let stmt = match inner.as_rule() {
                    Rule::variable_statement => as_var_assignment(inner),
                    Rule::if_statement => as_if_statment(inner),
                    Rule::print_statement => as_print_statement(inner),
                    Rule::expression_statement => Token::Statement(Statement::Expression(
                        as_expression(inner.into_inner().next().unwrap()),
                    )),
                    Rule::while_statement => as_while_statement(inner),
                    _ => continue,
                };
                tokens.push(stmt);
            }
            Rule::expression => {
                tokens.push(Token::Expression(as_expression(pair)));
            }
            _ => unreachable!(),
        }
    }
    tokens
}

pub fn evaluate_num_operations(pair: Pair<Rule>) -> Expression {
    let climber = PrecClimber::new(vec![
        Operator::new(Rule::plus, Assoc::Left) | Operator::new(Rule::minus, Assoc::Left),
        Operator::new(Rule::mul, Assoc::Left)
            | Operator::new(Rule::div, Assoc::Left)
            | Operator::new(Rule::mod_op, Assoc::Left),
        Operator::new(Rule::pow, Assoc::Right),
    ]);
    consume_bin_num_op(pair, &climber)
}

fn consume_bin_num_op(pair: Pair<Rule>, climber: &PrecClimber<Rule>) -> Expression {
    let primary = |pair| consume_bin_num_op(pair, climber);

    let infix = |left: Expression, op: Pair<Rule>, right: Expression| match op.as_rule() {
        Rule::plus => Expression::BinaryOp(Box::new(BinaryOp::Plus { left, right })),
        Rule::minus => Expression::BinaryOp(Box::new(BinaryOp::Minus { left, right })),
        Rule::mod_op => Expression::BinaryOp(Box::new(BinaryOp::Mod { left, right })),
        Rule::mul => Expression::BinaryOp(Box::new(BinaryOp::Mul { left, right })),
        Rule::div => Expression::BinaryOp(Box::new(BinaryOp::Div { left, right })),
        Rule::pow => Expression::BinaryOp(Box::new(BinaryOp::Pow { left, right })),
        p => Expression::Atomic(Atomic::String(format!("{:?}", p))),
    };

    match pair.as_rule() {
        Rule::binary_num_expression => climber.climb(pair.into_inner(), primary, infix),
        Rule::parent_expression => consume_bin_num_op(pair.into_inner().next().unwrap(), climber),
        Rule::rvalue_maybe_numeric => get_literal(pair.into_inner()),
        p => unreachable!("{:?}", p),
    }
}

fn as_print_statement(inner: Pair<Rule>) -> Token {
    let mut inner = inner.into_inner().skip(1);
    let string = as_expression(inner.next().unwrap());
    Token::Statement(Statement::Print(string))
}

fn as_if_statment(inner: Pair<Rule>) -> Token {
    let mut inner = inner.into_inner().skip(1);
    let c = inner.next().unwrap().into_inner().next().unwrap();
    let condition = as_expression(c);
    let body = parse_body(inner.next().unwrap());
    let mut else_body = None;
    if inner.next().is_some() {
        else_body = Some(parse_body(inner.next().unwrap()));
    }
    Token::Statement(Statement::Conditional(Conditional {
        condition: Box::new(condition),
        body,
        else_body,
    }))
}

fn as_while_statement(inner: Pair<Rule>) -> Token {
    let mut inner = inner.into_inner().skip(1);
    let c = inner.next().unwrap().into_inner().next().unwrap();
    let condition = as_expression(c);
    let body = parse_body(inner.next().unwrap());
    Token::Statement(Statement::Loop(Loop {
        condition: Box::new(condition),
        body,
    }))
}

fn as_var_assignment(pair: Pair<Rule>) -> Token {
    let inner = pair.into_inner().collect::<Vec<_>>();
    let is_new_var = matches!(inner[0].as_rule(), Rule::let_name);
    let ident_name = if is_new_var {
        inner[1].as_str()
    } else {
        inner[0].as_str()
    };
    let expression = if is_new_var {
        inner[2].clone()
    } else {
        inner[1].clone()
    };
    let expression = as_expression(expression);
    let stmt = Statement::VariableAssignment(VariableAssignment {
        new_definition: is_new_var,
        ident: ident_name.to_string(),
        value: expression,
    });
    Token::Statement(stmt)
}

fn as_expression(expression: Pair<Rule>) -> Expression {
    match expression.as_rule() {
        Rule::if_expression => as_if_expression(expression),
        Rule::binary_num_expression => as_binary_number_expression(expression),
        Rule::boolean_operation => as_boolean_operation(expression),
        Rule::boolean_expression => as_boolean_expression(expression),
        Rule::rvalue => as_literal(expression),
        Rule::binary_string_expression => todo! {"Not implemented"},
        Rule::expression => {
            let mut inner = expression.into_inner();
            let expr = inner.next().unwrap();
            if let Some(cast) = inner.next() {
                as_cast(cast, as_expression(expr))
            } else {
                as_expression(expr)
            }
        }
        Rule::input_expression => as_input_expression(expression.into_inner().nth(1).unwrap()),
        _ => unreachable!("{:?}", expression),
    }
}
fn as_cast(cast: Pair<Rule>, expr: Expression) -> Expression {
    let inner = cast
        .into_inner()
        .nth(1)
        .unwrap()
        .into_inner()
        .next()
        .unwrap();
    match inner.as_rule() {
        Rule::string => Expression::Cast(Box::new(Cast::String(expr))),
        Rule::number => Expression::Cast(Box::new(Cast::Int(expr))),
        Rule::bool => Expression::Cast(Box::new(Cast::Bool(expr))),
        _ => unreachable!(),
    }
}

fn as_input_expression(expression: Pair<Rule>) -> Expression {
    Expression::Input(Box::new(as_expression(expression)))
}

fn as_if_expression(inner: Pair<Rule>) -> Expression {
    let mut inner = inner.into_inner().skip(1);
    let condition = as_expression(inner.next().unwrap().into_inner().next().unwrap());
    let body = parse_body(inner.next().unwrap());
    let body_expression = as_expression(inner.next().unwrap().into_inner().next().unwrap());
    inner.next();
    let mut else_body = parse_body(inner.next().unwrap());
    let mut else_expression = Box::new(as_expression(
        inner.next().unwrap().into_inner().next().unwrap(),
    ));

    Expression::Conditional(ConditionalExpression {
        condition: Box::new(condition),
        body,
        body_expression: Box::new(body_expression),
        else_body,
        else_body_expression: else_expression,
    })
}

fn as_binary_number_expression(pair: Pair<Rule>) -> Expression {
    evaluate_num_operations(pair)
}

fn as_boolean_operation(pair: Pair<Rule>) -> Expression {
    let mut inner = pair.into_inner();
    let lhs = inner.next().unwrap();
    let op = inner.next().unwrap();
    let rhs = inner.next().unwrap();
    let lhs = match lhs.as_rule() {
        Rule::rvalue => as_literal(lhs),
        Rule::boolean_expression => as_boolean_expression(lhs),
        _ => unreachable!(),
    };
    let rhs = match rhs.as_rule() {
        Rule::rvalue => as_literal(rhs),
        Rule::boolean_expression => as_boolean_expression(rhs),
        _ => unreachable!(),
    };
    match op.as_rule() {
        Rule::and => Expression::LogicOp(Box::new(LogicOp::And(lhs, rhs))),
        Rule::or => Expression::LogicOp(Box::new(LogicOp::Or(lhs, rhs))),
        _ => unreachable!(),
    }
}

fn as_boolean_expression(pair: Pair<Rule>) -> Expression {
    let mut inner = pair.into_inner();
    let lhs = as_literal(inner.next().unwrap());
    let comparison = inner.next().unwrap();
    let rhs = as_literal(inner.next().unwrap());
    as_comparison(lhs, rhs, comparison)
}

fn as_comparison(lhs: Expression, rhs: Expression, pair: Pair<Rule>) -> Expression {
    match pair.into_inner().next().unwrap().as_rule() {
        Rule::smaller_than => Expression::Comparison(Box::new(Comparison::Smaller(lhs, rhs))),
        Rule::smaller_equals => {
            Expression::Comparison(Box::new(Comparison::SmallerEquals(lhs, rhs)))
        }
        Rule::equals => Expression::Comparison(Box::new(Comparison::Equals(lhs, rhs))),
        Rule::greater_equals => {
            Expression::Comparison(Box::new(Comparison::GreaterEquals(lhs, rhs)))
        }
        Rule::greater_than => Expression::Comparison(Box::new(Comparison::Greater(lhs, rhs))),
        _ => unreachable!(),
    }
}

fn as_literal(pair: Pair<Rule>) -> Expression {
    let inner = pair.into_inner().next().unwrap();
    match inner.as_rule() {
        Rule::boolean_literal => match inner.into_inner().next().unwrap().as_rule() {
            Rule::true_literal => Expression::Atomic(Atomic::Bool(true)),
            Rule::false_literal => Expression::Atomic(Atomic::Bool(false)),
            _ => unreachable!(),
        },
        Rule::string_literal => {
            let inner = inner.into_inner().nth(1).unwrap().as_str().to_string();
            Expression::Atomic(Atomic::String(inner))
        }
        Rule::variable_name => Expression::Ident(inner.as_str().to_string()),
        Rule::numeric_literal => {
            Expression::Atomic(Atomic::Number(inner.as_str().trim().parse().unwrap()))
        }
        Rule::expression => as_expression(inner),
        p => unreachable!("{:?}", p),
    }
}

fn get_literal(mut pair: Pairs<Rule>) -> Expression {
    let element: Pair<Rule> = pair.next().unwrap();
    match element.as_rule() {
        Rule::numeric_literal => Expression::Atomic(Atomic::Number(
            element.as_str().trim().parse::<f64>().unwrap(),
        )),
        Rule::variable_name => Expression::Ident(element.as_str().to_string()),
        p => {
            println!("{:?}", p);
            unreachable!()
        }
    }
}


Wir kommen nun nochmals zur Variable am Anfang zurück. Das Result haben wir bereits geparsed, jetzt wollen wir das `pest` Resultat in einen AST umwandeln. Führe folgenden Code aus.

> Wir müssen ein paar mal `next()` und `unwrap()` aufrufen, um zum `body` unserer Grammatik zu kommen. Schau dir die Gramamtik (`ubpp.pest`) an und versuche zu verstehen, was für eine Regel bei jeder Zeile stehen sollte.



In [35]:
// Für iterators müssen wir unser Resultat mutably definieren
// Wir shadowen nehmen Ownership und deklarieren die neue Version als mutable.
let mut parse_result = parse_result;
let parse_result = parse_result.next()
                        .unwrap()
                        .into_inner()
                        .next()
                        .unwrap();
let tokens = parse_body(parse_result);
println!("{:#?}", tokens);

[
    Statement(
        VariableAssignment(
            VariableAssignment {
                new_definition: true,
                ident: "priimZahl",
                value: Input(
                    Atomic(
                        String(


                            "Was füre Zahl wotsch teste?",
                        ),
                    ),
                ),
            },
        ),
    ),
]


## AST evaluieren (Step3)

### Results

Wir haben bereits `Option<T>` angeschaut. Für diesen Typen war die Argumentation, dass ein Wert da sein kann, oder eben nicht (was in vielen Sprachen durch nullability ausgedrückt wird). Was aber wenn eigentlich immer ein Wert da sein soll, ausser es ist ein Fehler passiert?

Dafür gibt es in Rust den `Result<T,E>` Typen, der im Wesentlichen so wie folgt definiert werden kann:

```rust
enum Result<T,E> {
    Ok(T),
    Err(E)
}
```

Dies bedeutet, dass man im Erfolgsfall den Typen `T` als `Ok` Wert zurückgibt, und falls etwas schief läuft, mit `Err` den Typen `E` zurückgeben kann. Der Typ `E` kann im Prinzip irgendein Typ sein, wobei es allerdings einen `Error`-Trait gibt, der bestimmte Vorteile bringt. Für uns ist das vorerst nicht wirklich von Interesse und wir werden im Normalfall einfachheitshalber mit einem `String` als Fehler-Typen arbeiten.

Wie auch für `Option<T>` sind verschiedene Funktionen aus der Standardbibliothek verfügbar. Zusätzlich zu den bekannten, gibt es noch die enstprechenden Funktionen um den `Error` Typen zu modifizieren.

Das Resultat oder den Fehler können wir mit dem Pattern-Matching erreichen, oder mit dem bekannten `unwrap()` oder `unwrap_err()`.

### Bubbling

Wenn eine Funktion den gleichen Error Typen im Result-Fall oder den gleichen Option Typ verwendet, wie die Funktion, die aufgerufen wird, kann man einfachheitshalber mit dem `?`-Operator den Fehler (oder das Fehlen) nach oben "bubbeln". Dies bedeutet, wir müssen den Non-Happy-Case nicht explizit in der Funktion handeln, sondern überlassen das Fehlerhandling der Parent-Funktion. Dies vereinfacht das Lesen des Codes und erlaubt es uns, das Fehlerhandling zentral zu erledigen. Dies ist insbesondere für Bibliotheken von Interesse, da wir vielleicht das Fehler-Handling der Applikation überlassen wollen.

In [49]:
fn can_fail() -> Result<bool, String> {
    let test_string = deeper_fail("test")?;
    Ok(test_string == "test")
}

// Der Error-Typ (sprich der String) ist gleich wie bei `can_fail`.
fn deeper_fail(t : &str) -> Result<String, String> {
    if t == "test" {
        Ok(String::from("true"))
    } else {
        Err(String::from("Ohoh ein Fehler"))
    }
}

match can_fail() {
    Ok(bool) => println!("{}", bool),
    Err(err_string) => println!("Ein Fehler ist passiert: {}", err_string)
}

false


()

### Klassenmethoden

In Rust werden Methoden von Structs nicht in der Struct-Definition definiert. Sie können sogar in einem beliebigen Kontext hinzugefügt werden (solange der Typ im gleichen Crate definiert wird). Dafür benutzt man den `impl Structname` Syntax. 

Ähnlich zu Python kann als erstes Argument `self` in einer der drei Varianten stehen (`self`, `&self`, `&mut self`), was die Funktion zu einer "Instanz-Funktion" macht. Die Funktion kann via desugaring als `struct_instance.function()` aufgerufen werden, was aber im Prinzip nur `Struct::function(&struct_instance)` (resp. dem entsprechenden `self` Argument) entspricht.

Innerhalb des `impl` Blocks, kann man auf das Keyword `Self` (man beachte das grosse `S`) zurückgreifen, was einfach zum Namen der Struct desugared. Dies ist insofern sinnvoll, als dass Refactorings der Struct einfacher werden können, sollte man keinen Language-Server-Support haben. Weiter drückt es unmissverständlich aus, dass eine Instanz von der Struct, für die der `impl`-Block steht, erzeugt wird. Am Ende ist dies aber natürlich eine Stil-Frage.

In [56]:
struct SomethingSpecial {
    element: i32
}
impl SomethingSpecial {
    // dies ist eine statische Funktion
    pub fn new(element: i32) -> Self {
        // whooooooa whats happening here?
        // Wir benutzen den Struct-Constructor (wie schon gezeigt), aber weil unser Argument
        // der Funktion `new` gleich heisst, wie das Feld, können wir das in der 
        // abgekürzten Form schreiben
        // 
        // Weiter verwenden wir überall wo wir `SomethingSpecial` schreiben könnten `Self`
        Self {
            element
        }
    }

    // wollen wir Daten in  der Struct ändern, brauchen wir eine mutable Reference....
    pub fn increase(&mut self) {
        self.element += 1;
    }

    //... oder eine owning Funktion
    // vorsicht hier, ohne den return, würde `self` consumed (es kann immer nur jemand Ownership von einem Objekt haben)
    pub fn increase_consuming(mut self) -> Self {
        self.element += 1;
        self
    }
    // ohne getter und ohne `pub` Keyword in der Struct ist unser Feld nur innerhalb des gleichen Moduls
    // accessable.
    pub fn get_element(&self) -> i32 {
        self.element
    }

}

let something_special = SomethingSpecial::new(0);
// warum geht das nicht?
something_special.increase();
// hier wird something_special consumed
something_special.increase_consuming().increase_consuming().increase_consuming();
// wie gross ist jetzt element?
println!("{}", something_special.get_element());

Error: cannot borrow `something_special` as mutable, as it is not declared as mutable

Error: borrow of moved value: `something_special`

Wir sind nun bereit, uns den Code für die Implementierung auf der Expression anzuschauen. Wir benutzen Klassen-Methoden um eine Expression als Atomic-Datentypen zu evaluieren. Da dies aber potenziel nicht funktionieren kann (zum Beispiel wenn wir versuchen eine nicht Atomic Expression zu casten), geben wir ein Result zurück.

In unserer Sprache wollen wir so flexibel wie möglich sein was Daten-Typen anbelangt, und implementieren entsprechend für alle Atomics einen Cast zu allen anderen Atomic-Datatypes.

So kann in einem if-Block z.b. "false" benutzt werden, weil "false" als Boolean Typ geparsed wird.

Der einfachheitshalber benutzen wir nur einen einzigen globalen Scope um Variabeln zu referenzieren. Dies führt in bestimmten Fällen zu verwirrenden Situationen, wenn Variabeln mit dem gleichen Namen in Strukturen wie Loops, oder Conditionals evaluiert werden. Wir als gewissenhafte Programmierer brauchen aber hier keine Hilfe, weil wir immer sinnvolle und verschiedene Variabelnnamen benutzten ;)

In [74]:
impl Expression {
    fn as_bool(&self, global_scope: &mut HashMap<String, Expression>) -> Result<bool, String> {
        let inner = eval_expression(self, global_scope)?;
        match inner {
            Atomic::String(s) => Ok(s.parse().map_err(|e| format!("{:?}", e))?),
            Atomic::Number(i) => Ok(i == 0.0),
            Atomic::Bool(b) => Ok(b),
            Atomic::Null => Ok(false),
            Atomic::Interrupt => unreachable!(),
        }
    }
    fn as_string(&self, global_scope: &mut HashMap<String, Expression>) -> Result<String, String> {
        let inner = eval_expression(self, global_scope)?;
        match inner {
            Atomic::String(s) => Ok(s),
            Atomic::Number(i) => Ok(i.to_string()),
            Atomic::Bool(b) => Ok(b.to_string()),
            Atomic::Null => Ok("null".to_string()),
            Atomic::Interrupt => unreachable!(),
        }
    }
    fn as_num(&self, global_scope: &mut HashMap<String, Expression>) -> Result<f64, String> {
        let inner = eval_expression(self, global_scope)?;
        match inner {
            Atomic::String(s) => Ok(s.trim().parse().map_err(|e| format!("{:?}", e))?),
            Atomic::Number(i) => Ok(i),
            Atomic::Bool(b) => Ok(b as i32 as f64),
            Atomic::Null => Ok(0.0),
            Atomic::Interrupt => unreachable!(),
        }
    }
}

Error: the usage of Script Group `Greek` in this crate consists solely of mixed script confusables

Error: duplicate definitions with name `as_bool`

Error: duplicate definitions with name `as_string`

Error: duplicate definitions with name `as_num`

Wir verwenden für das evaluieren der Tokens einen ähnlichen Ansatz, wie beim erstellen des ASTs. Wir benutzen eine Schleife über alle Tokens und führen sie der Reihenfolge entsprechend aus.

Wir verwenden den Typen `&[Token]`, um alle Referenzen die als Array (Slice) gecasted werden können zu unterstützen. So müssen wir uns z.B. nicht auf `&Vec<T>` einschränken, sondern unterstützen sämtliche Typen, die `AsRef<[Token]>` implementieren.

In [ ]:
pub fn eval_tokens(
    tokens: &[Token],
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    let mut last_expression = Atomic::Null;
    for token in tokens {
        match token {
            Token::Expression(e) => {
                last_expression = eval_expression(e, global_scope)?;
                // wir verwenden hier ein Macro um den Match einfacher lesbar zumachen
                // Dies desugared zu einem einfachen let Statement.
                if matches!(last_expression, Atomic::Interrupt) {
                    return Ok(last_expression);
                }
            }
            Token::Statement(stmt) => {
                let is_interrupt = eval_statement(stmt, global_scope)?;
                if matches!(is_interrupt, Atomic::Interrupt) {
                    return Ok(is_interrupt);
                }
            }
            Token::Break => {
                return Ok(Atomic::Interrupt);
            },
            Token::Return => return Ok(Atomic::Interrupt),
        }
    }
    Ok(last_expression)
}

Der Aufbau hier ist wieder sehr ähnlich zum Step2. Wir versuchen immer spezifischere Funktionen anzubieten, um ein Rekursives evaluieren vom AST zu ermöglichen. Darum schauen wir uns jeweils nur ein Beispiel von `eval_statement` und `eval_expression` genauer an.

### Borrowing

Das erste Beispiel, das wir anschauen ist `eval_assignment`. Wir wollen sicherstellen, dass eine Variable nicht zweimal definiert werden kann. Dafür nutzen wir die Funktion `contains_key` der HashMap. Weil wir nun auf den Value innerhalb von `assignment` zugreifen wollen, müssen wir schauen, dass wir den Wert nicht im falschen "Modus" benutzen. Da wir hier `assignment` nur leihen, können wir nichts anderes machen, als alle Felder innerhalb von `assignment` ebenfalls zu leihen.

Darum müssen wir zwingend `&assignment.ident` verwenden, abgesehen davon, dass `contains_key` sowieso einen Borrow will.

Falls also unser Variablenassignment, gemäss unserer Grammatik, eine neue Definition darstellt, vergewissern wir uns, dass die entsprechende Variable noch __nicht__ definiert wurde.

Ansonsten nutzen wir die `insert` Funktion, die den Wert einfügt oder updated (also eigentlich `upsert`).

Falls alles klappt, signalisieren wir dies mit `Ok(())`, wobei `()` der sogenannte `Unit`-Typ ist, und in etwa equivalent zum C-Type `void`.

In [ ]:
fn eval_assignment(
    assignment: &VariableAssignment,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<(), String> {
    if assignment.new_definition && global_scope.contains_key(&assignment.ident) {
        return Err(format!("`{}` already defined", assignment.ident));
    } else {
        let result = eval_expression(&assignment.value, global_scope)?;
        global_scope.insert(assignment.ident.to_string(), Expression::Atomic(result));
    }
    Ok(())
}

Als Beispiel für die Evaluierung einer Expression, schauen wir uns die Evaluierung einer binären Operation, als einer Operation mit zwei Operanden, an. Wie so oft in den anderen Beispielen, sehen wir auch hier ein Pattern-Matching. Im Falle der binären Operation haben wir den Struct-Enum-Variant benutzt, der sich hier nun wieder zeigt.

Das Matching hier sieht vorerst genau gleich aus, wie bei den Tuple-Varianten, ausser, dass, anstelle von `()` `{}` stehen um auf die inneren Elemente des Enum-Variant zuzugreifen. Dies wird aber einfach vom Fakt überdeckt, dass man auch beim Pattern-Matching auf den Struct-Shorthand zurückgreifen kann. Eigentlich würde das Struct-Matching wie folgt aussehen:

```rust
match num_op {
    BinaryOp::Plus {left: left, right: right} => {...}
}
```

Dies zeigt uns einen der Vorteile der Struct-Enum-Variant: die Argumente sind benannt!

In [ ]:
fn eval_binary_op(
    num_op: &BinaryOp,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    match num_op {
        BinaryOp::Plus { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left + right))
        }
        BinaryOp::Minus { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left - right))
        }
        BinaryOp::Mul { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left * right))
        }
        BinaryOp::Div { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left / right))
        }
        BinaryOp::Mod { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left % right))
        }
        BinaryOp::Pow { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left.powf(right)))
        }
        BinaryOp::None => unreachable!(),
    }
}

Im folgenden Listing ist nun der gesammte evaluierungs Code. Schaue nochmals durch ob du alles verstehst und führe ihn anschliessend aus:

In [76]:
use std::{collections::HashMap, io::stdin};

impl Expression {
    fn as_bool(&self, global_scope: &mut HashMap<String, Expression>) -> Result<bool, String> {
        let inner = eval_expression(self, global_scope)?;
        match inner {
            Atomic::String(s) => Ok(s.parse().map_err(|e| format!("{:?}", e))?),
            Atomic::Number(i) => Ok(i == 0.0),
            Atomic::Bool(b) => Ok(b),
            Atomic::Null => Ok(false),
            Atomic::Interrupt => unreachable!(),
        }
    }
    fn as_string(&self, global_scope: &mut HashMap<String, Expression>) -> Result<String, String> {
        let inner = eval_expression(self, global_scope)?;
        match inner {
            Atomic::String(s) => Ok(s),
            Atomic::Number(i) => Ok(i.to_string()),
            Atomic::Bool(b) => Ok(b.to_string()),
            Atomic::Null => Ok("null".to_string()),
            Atomic::Interrupt => unreachable!(),
        }
    }
    fn as_num(&self, global_scope: &mut HashMap<String, Expression>) -> Result<f64, String> {
        let inner = eval_expression(self, global_scope)?;
        match inner {
            Atomic::String(s) => Ok(s.trim().parse().map_err(|e| format!("{:?}", e))?),
            Atomic::Number(i) => Ok(i),
            Atomic::Bool(b) => Ok(b as i32 as f64),
            Atomic::Null => Ok(0.0),
            Atomic::Interrupt => unreachable!(),
        }
    }
}

pub fn eval_tokens(
    tokens: &[Token],
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    let mut last_expression = Atomic::Null;
    for token in tokens {
        match token {
            Token::Expression(e) => {
                last_expression = eval_expression(e, global_scope)?;
                if matches!(last_expression, Atomic::Interrupt) {
                    return Ok(last_expression);
                }
            }
            Token::Statement(stmt) => {
                let is_interrupt = eval_statement(stmt, global_scope)?;
                if matches!(is_interrupt, Atomic::Interrupt) {
                    return Ok(is_interrupt);
                }
            }
            Token::Break => {
                return Ok(Atomic::Interrupt);
            },
            Token::Return => return Ok(Atomic::Interrupt),
        }
    }
    Ok(last_expression)
}

fn eval_statement(
    stmt: &Statement,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    match stmt {
        Statement::VariableAssignment(assignment) => {
            eval_assignment(assignment, global_scope)?;
            Ok(Atomic::Null)
        }
        Statement::Conditional(c) => eval_conditional(c, global_scope),
        Statement::Expression(e) => {
            let token = eval_expression(e, global_scope)?;
            Ok(token)
        }
        Statement::Print(expression) => {
            let result = eval_expression(expression, global_scope)?;
            // Wir haben nur debug derived... wie können wir wohl display implementieren?
            // println!("{:?}", result);
            println!("{}", result);
            Ok(result)
        }
        Statement::Loop(loop_statement) => eval_loop(loop_statement, global_scope),
    }
}

fn eval_loop(
    loop_statement: &Loop,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    let mut condition = loop_statement.condition.as_bool(global_scope)?;
    while condition {
        let token = eval_tokens(&loop_statement.body, global_scope)?;
        if matches!(token, Atomic::Interrupt) {
            return Ok(Atomic::Null);
        }
        condition = loop_statement.condition.as_bool(global_scope)?;
    }
    Ok(Atomic::Null)
}

fn eval_conditional(
    conditional: &Conditional,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    let condition = conditional.condition.as_bool(global_scope)?;
    if condition {
        let token = eval_tokens(&conditional.body, global_scope)?;
        Ok(token)
    } else if let Some(tokens) = conditional.else_body.as_ref() {
        let token = eval_tokens(tokens, global_scope)?;
        Ok(token)
    } else {
        Ok(Atomic::Null)
    }
}

fn eval_assignment(
    assignment: &VariableAssignment,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<(), String> {
    if assignment.new_definition && global_scope.contains_key(&assignment.ident) {
        return Err(format!("`{}` already defined", assignment.ident));
    } else {
        let result = eval_expression(&assignment.value, global_scope)?;
        global_scope.insert(assignment.ident.to_string(), Expression::Atomic(result));
    }
    Ok(())
}

fn eval_expression(
    e: &Expression,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    match e {
        Expression::Atomic(atomic) => Ok(atomic.to_owned()),
        Expression::Ident(ident) => {
            let ident_expression = if let Some(expr) = global_scope.get(ident) {
                expr.clone()
            } else {
                return Err(format!("[ERROR] `{}` not defined!", ident));
            };
            eval_expression(&ident_expression, global_scope)
        }
        Expression::Input(expression) => {
            let mut s = String::new();
            println!("{}", expression.as_string(global_scope)?);
            stdin().read_line(&mut s).map_err(|e| format!("{:?}", e))?;
            Ok(Atomic::String(s))
        }
        Expression::LogicOp(logic_operation) => eval_logic_op(logic_operation, global_scope),
        Expression::Comparison(comparison) => eval_comparison(comparison, global_scope),
        Expression::BinaryOp(num_op) => eval_binary_op(num_op, global_scope),
        Expression::Conditional(conditional) => {
            eval_conditional_expression(conditional, global_scope)
        }
        Expression::Cast(cast) => match cast.as_ref() {
            Cast::String(expr) => {
                let result = expr.as_string(global_scope)?;
                Ok(Atomic::String(result))
            }
            Cast::Int(expr) => {
                let result = expr.as_num(global_scope)?;
                Ok(Atomic::Number(result))
            }
            Cast::Bool(expr) => {
                let result = expr.as_bool(global_scope)?;
                Ok(Atomic::Bool(result))
            }
        },
    }
}

fn eval_conditional_expression(
    conditional: &ConditionalExpression,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    let condition = conditional.condition.as_bool(global_scope)?;
    if condition {
        let _ = eval_tokens(&conditional.body, global_scope);
        eval_expression(&conditional.body_expression, global_scope)
    } else {
        let _ = eval_tokens(&conditional.else_body, global_scope);
        eval_expression(&conditional.else_body_expression, global_scope)
    }
}

fn eval_logic_op(
    logic_operation: &LogicOp,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    match logic_operation {
        LogicOp::And(lhs, rhs) => {
            let lhs = lhs.as_bool(global_scope)?;
            let rhs = rhs.as_bool(global_scope)?;
            Ok(Atomic::Bool(lhs && rhs))
        }
        LogicOp::Or(lhs, rhs) => {
            let lhs = lhs.as_bool(global_scope)?;
            let rhs = rhs.as_bool(global_scope)?;
            Ok(Atomic::Bool(lhs || rhs))
        }
    }
}

fn eval_comparison(
    comparison: &Comparison,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    match comparison {
        Comparison::Smaller(lhs, rhs) => {
            if let (Ok(l), Ok(r)) = (lhs.as_num(global_scope), rhs.as_num(global_scope)) {
                Ok(Atomic::Bool(l < r))
            } else if let (Ok(l), Ok(r)) =
                (lhs.as_string(global_scope), rhs.as_string(global_scope))
            {
                Ok(Atomic::Bool(l < r))
            } else {
                Err(format!("Invalid operands to comparison ({:?})", comparison))
            }
        }
        Comparison::SmallerEquals(lhs, rhs) => {
            if let (Ok(l), Ok(r)) = (lhs.as_num(global_scope), rhs.as_num(global_scope)) {
                Ok(Atomic::Bool(l <= r))
            } else if let (Ok(l), Ok(r)) =
                (lhs.as_string(global_scope), rhs.as_string(global_scope))
            {
                Ok(Atomic::Bool(l <= r))
            } else {
                Err("Invalid operands to comparison".to_string())
            }
        }
        Comparison::Equals(lhs, rhs) => {
            if let (Ok(l), Ok(r)) = (lhs.as_num(global_scope), rhs.as_num(global_scope)) {
                Ok(Atomic::Bool(l == r))
            } else if let (Ok(l), Ok(r)) =
                (lhs.as_string(global_scope), rhs.as_string(global_scope))
            {
                Ok(Atomic::Bool(l == r))
            } else {
                Err("Invalid operands to comparison".to_string())
            }
        }
        Comparison::Greater(lhs, rhs) => {
            if let (Ok(l), Ok(r)) = (lhs.as_num(global_scope), rhs.as_num(global_scope)) {
                Ok(Atomic::Bool(l > r))
            } else if let (Ok(l), Ok(r)) =
                (lhs.as_string(global_scope), rhs.as_string(global_scope))
            {
                Ok(Atomic::Bool(l > r))
            } else {
                Err("Invalid operands to comparison".to_string())
            }
        }
        Comparison::GreaterEquals(lhs, rhs) => {
            if let (Ok(l), Ok(r)) = (lhs.as_num(global_scope), rhs.as_num(global_scope)) {
                Ok(Atomic::Bool(l >= r))
            } else if let (Ok(l), Ok(r)) =
                (lhs.as_string(global_scope), rhs.as_string(global_scope))
            {
                Ok(Atomic::Bool(l >= r))
            } else {
                Err("Invalid operands to comparison".to_string())
            }
        }
    }
}

fn eval_binary_op(
    num_op: &BinaryOp,
    global_scope: &mut HashMap<String, Expression>,
) -> Result<Atomic, String> {
    match num_op {
        BinaryOp::Plus { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left + right))
        }
        BinaryOp::Minus { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left - right))
        }
        BinaryOp::Mul { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left * right))
        }
        BinaryOp::Div { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left / right))
        }
        BinaryOp::Mod { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left % right))
        }
        BinaryOp::Pow { left, right } => {
            let left = left.as_num(global_scope)?;
            let right = right.as_num(global_scope)?;
            Ok(Atomic::Number(left.powf(right)))
        }
        BinaryOp::None => unreachable!(),
    }
}


Error: the usage of Script Group `Greek` in this crate consists solely of mixed script confusables

Error: duplicate definitions with name `as_bool`

Error: duplicate definitions with name `as_string`

Error: duplicate definitions with name `as_num`

## Erster Test mit unserer Sprache

Soweit so gut. Nun haben wir alle Elemente die wir brauchen, um Programme laufen zu lassen.

Wir haben zuerst unser Datenmodel definiert. Diese Struktur haben wir unseren AST genannt, und sie sollte eine Struktur darstellen, die einfach ist zu evaluieren.

Als nächstes haben wir angefangen Code zu schreiben, der das Resultat von `pest` (diese Pairs von Rules) verarbeitet hat, und in unseren AST umgewandelt hat. 

Zu guter letzt haben wir uns an die Evaluierung unseres AST gesetzt und sind nun am Punkt, wo wir ein erstes Programm ausführen können.

In [61]:
// unser Program-Code, in diesem Beispiel benutzen wir den Primzahltester, aber wir können nun beliebige Programme erstellen.
let unser_code = r#"
definier e variable wo priimZahl heisst mit em wert 10239;
definier e variable wo teiler heisst mit em wert 2;
definier e variable wo primzahlWurzel heisst mit em wert priimZahl hoch 0.5;
definier e variable wo derest heisst mit em wert -1;
definier e variable wo priimZahlGfunde heisst mit em wert wohr;

solang de teiler kliiner oder gliich isch wie primzahlWurzel mach {
    falls de (priimZahl rest teiler) gliich isch wie 0 mach {
        priimZahlGfunde isch falsch;
        gib us ("Zum bispil isch: ");
        gib us (teiler);
        gib us ("en Teiler");
        stop;
    }
    teiler isch teiler plus 1;
}

falls d priimZahlGfunde mach {
    gib us ("S isch e priim zahl");
} suscht {
    gib us ("S isch also kei priim zahl");
};
"#;

// Wir parsen unseren Code mit pest
let mut parse_result = UBPP::parse(Rule::file, unser_code).unwrap();
//Unser Result wird nun in unseren AST umgewandelt..
let parse_result = parse_result.next()
                        .unwrap()
                        .into_inner()
                        .next()
                        .unwrap();
let tokens = parse_body(parse_result);

// ... wir erstellen einen Scope für unser Program
// diesen könnten wir z.B. auch benutzen um dem Programm ARgumente zu übergeben
let mut global_tokens = HashMap::new();
// Anschliessend wird es ausgeführt.
let result = eval_tokens(&tokens, &mut global_tokens).unwrap();

// Das Result in unserem Primzahltester sollte immer Null sein, da kein Resultat erwartet wird (der letzte Block ist ein Statement)
println!("Das Program wurde erfolgreich ausgeführt. Der Returnwert ist {:?}", result);

String("Zum bispil isch:")
Number(3.0)
String("en Teiler")
String("S isch also kei priim zahl")
Das Program wurde erfolgreich ausgeführt. Der Returnwert ist Null


Noice! Das funktioniert sogar! Es ist jetzt nur noch etwas uncool, dass wir den Debug output beim "print" Statement benutzen.

```rust
(...)

Statement::Print(expression) => {
    let result = eval_expression(expression, global_scope)?;
    // Wir haben nur debug derived... wie können wir wohl display implementieren?
    println!("{:?}", result);
    // println!("{}", result);
    Ok(result)
}

(...)
```

Wir können das besser machen. Wir haben anfangs mal erwähnt, dass es verschiedene Traits (Interfaces, Protokolle) gibt, die benutzt werden, um den Debug-String resp. den Display-String darzustellen. Wir kommen somit zum letzten Thema für heute.

## Traits

Wir kennen nun bereits den Syntax, um Funktionen für Structs bereitzustellen. Die Implementierung eines Trait funktioniert sehr ähnlich. Einzig die Funktionen dürfen wir nun nicht mehr selber wählen, sondern sind durch die Traits bestimmt.

Wir definieren mal ein einfaches Trait für uns. Traits besitzen Funktionen die implementiert werden müssen. Eine Trait-Definition darf auch "default" Funktionsimplementierungen bereitstellen, die benutzt werden, falls ein Implementer diese nicht überschreibt.

> Rust Identifiers dürfen (fast) beliebige UTF8-Characters sein. Wir können also endlich unsere Umlaute und Griechischen Buchstaben benutzen!

In [62]:

// Unser Trait
pub trait Tier {
    // in einem Trait können wir z.B. auch die explizite Implementierung haben
    type Art;
    // Dies ist eine statische Trait-Funktion
    fn name() -> String;
    // Hier haben wir eine Instanzfunktion.
    fn mach_geräusch(&self);
    fn mate(&self, other: &Self::Art) -> Self::Art;
    // Dies stellt eine Default Implementierung dar. Implementers müssen diese nicht implementieren.
    fn laufe(&self) {
        println!("*quiiiek*");
    }
}

pub fn π() -> f32 {
    std::f32::consts::PI
}

// Unsere zwei Tiere, Katze und Hund
#[derive(Debug)]
struct Katze {
    name: String,
}
#[derive(Clone, Debug)]
struct Hund {}

Wir wollen nun das Trait für die beiden Structs implementieren. Dafür verwenden wir den `impl ... for ...` Syntax.

In [63]:

// Unsere Trait Implementierung
impl Tier for Katze {
    type Art = Katze;
    fn name() -> String {
        String::from("Katze")
    }
    fn mach_geräusch(&self) {
        println!("miau");
    }

    fn mate(&self, other: &Self::Art) -> Self::Art {
        Katze {
            name: format!("{}<->{}", self.name, other.name),
        }
    }
}

impl Tier for Hund {
    type Art = Hund;

    fn name() -> String {
        String::from("Hund")
    }
    fn mach_geräusch(&self) {
        println!("wau");
    }

    fn mate(&self, other: &Self::Art) -> Self::Art {
        other.clone()
    }
}

Wir können nun auf den Trait als Trait-Bound für Funktionen verwenden

In [68]:
pub fn schlachte<T: Tier>(_tier: T) {
    println!("*katschak*: {} tot", T::name());
}

pub fn pflege<T>(_tier: T) where T: Tier {
    println!("*petpet*");
}

Und schauen uns den Output mal an

In [69]:
let kater = Katze {
        name: "Kater".to_string(),
    };
    let katze = Katze {
        name: "Katze".to_string(),
    };
    let hunderich = Hund {};

    // zuerst katzen
    kater.mach_geräusch();
    katze.mach_geräusch();
    katze.laufe();

    let kind = kater.mate(&katze);
    println!("Das Kind heisst: {}", kind.name);

    hunderich.mach_geräusch();
    hunderich.laufe();

    // Warum geht das nicht?
    hunderich.mate(&katze);

    schlachte(kater);
    schlachte(hunderich);
    // Was ist hier das Problem?
    kater.laufe();

Error: the usage of Script Group `Greek` in this crate consists solely of mixed script confusables

Error: mismatched types

Lange Rede kurzer Sinn, wir können nun unsere Display-Implementierung für unseren Typen definieren


In [71]:
use std::fmt::Display;

impl Display for Atomic {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        match self {
            Atomic::String(s) => f.write_str(s),
            Atomic::Number(n) => f.write_str(&n.to_string()),
            Atomic::Bool(b) => f.write_str(&b.to_string()),
            Atomic::Null => f.write_str("null"),
            Atomic::Interrupt => f.write_str("<< interrupt >>"),
        }
    }
}

Error: the usage of Script Group `Greek` in this crate consists solely of mixed script confusables

Error: conflicting implementations of trait `std::fmt::Display` for type `Atomic`

# Schlusswort

Wir sind nun am Schluss des Tutorials. Wenn man sich sicher fühlt, kann man nun anfangen, den eigenen Dialekt in die Grammatik zu übernehmen. Weiter ist im Moment String Concatenation nicht definiert. Dies wäre eine gute Excercise, um das eigene Rust-Wissen zu überprüfen!
